In [27]:
import numpy as np
import random

def softmax(x):

    ### YOUR CODE HERE
    
    if(len(x.shape) > 1): # matrix
        rowmax = x.max(axis = 1, keepdims = True)
        x = x - rowmax
        exp = np.exp(x)
        rowsum = exp.sum(axis = 1,keepdims = True)
        return exp / rowsum
        
        
    else:  # vector
        maxval = x.max()
        x = x - maxval
        exp = np.exp(x)
        sums = exp.sum()
        return exp / sums
      

In [28]:
import numpy as np

def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    """
    deno = 1 + np.exp(-x)
   
    return 1. / deno

def sigmoid_grad(f):
    """
    Compute the gradient for the sigmoid function here. Note that
    for this implementation, the input f should be the sigmoid
    function value of your original input x. 
    """

    right = 1 - f
    return np.multiply(f, right)
    


In [29]:
def forward_backward_prop(data, labels, params, dimensions):
    """ 
    Forward and backward propagation for a two-layer  network hidden is sigmoidal and outer is softmax
    
    Compute the forward propagation and for the cross entropy cost,
    and backward propagation for the gradients for all parameters.
    """

    ### Unpack network parameters (do not modify)
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

    W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1, H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
    ofs += H * Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))

    ### YOUR CODE HERE: forward propagation
    
    hidden = sigmoid(data.dot(W1) + b1)
    prediction = softmax(hidden.dot(W2) + b2)
    
    
    ### END YOUR CODE
    
    ### YOUR CODE HERE: backward propagation
    raise NotImplementedError
    ### END YOUR CODE
    
    ### Stack gradients (do not modify)
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(), 
        gradW2.flatten(), gradb2.flatten()))
    
    return cost, grad

In [46]:
import numpy as np
import random

# First implement a gradient checker by filling in the following functions
def gradcheck_naive(f, x):
    """ 
    Gradient check for a function f 
    - f should be a function that takes a single argument and outputs the cost and its gradients
    - x is the point (numpy array) to check the gradient at
    """ 

    rndstate = random.getstate()
    random.setstate(rndstate)  
    fx, grad = f(x) # Evaluate function value at original point
    h = 1e-4

    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        ### try modifying x[ix] with h defined above to compute numerical gradients
        ### make sure you call random.setstate(rndstate) before calling f(x) each time, this will make it 
        ### possible to test cost functions with built in randomness later
        ### YOUR CODE HERE:
        x[ix] += h
        random.setstate(rndstate)  
        (costPos, d) = f(x)
        x[ix] -= 2*h
        random.setstate(rndstate)
        (costNeg, d) = f(x)
        numgrad = (costPos - costNeg) / (2. * h)
        x[ix] += h
        ### END YOUR CODE

        # Compare gradients
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print "Gradient check failed."
            print "First gradient error found at index %s" % str(ix)
            print "Your gradient: %f \t Numerical gradient: %f" % (grad[ix], numgrad)
            return
    
        it.iternext() # Step to next dimension

    print "Gradient check passed!"
